In [7]:
# Giai Đoạn 2 extract skill và Job từ description 

In [8]:
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)  # Fix lỗi từ trước

# Sample JD từ file của bạn
sample_jd ="""
We are looking for a supportive IT specialist to join our service desk team.
You will be the first point of contact for users reporting incidents, handling inquiries via phone, email, or ticketing system.
Responsibilities include diagnosing and resolving hardware/software issues quickly, escalating complex problems when needed,
and ensuring high customer satisfaction.
The ideal candidate has experience supporting Microsoft Windows environments, including Active Directory (AD), Exchange Server, and basic networking.
You should be able to multitask effectively in a fast-paced setting, remain calm when dealing with frustrated users,
and have excellent interpersonal skills to explain technical concepts clearly.
Fluency in Englisch is required; French is a plus.
Ability to adapt quickly to new tools and work well in a team-oriented environment is essential.
"""
# DB skill mẫu (từ O*NET/Lightcast + mở rộng)
skills_db = [
    'python', 'sql', 'java', 'javascript', 'excel', 'project management', 'communication', 'troubleshooting', 'networking',
    'Active Diractory', 'windows server', 'exchange', 'ticketing systems', 'english', 'french', 'information systems',
    'information technology', 'help desk', 'it support', 'machine learning', 'data analysis', 'toeic', 'ielts',
]

# Tokenize JD (NLP)
tokens = word_tokenize(sample_jd.lower())

# Extract bằng similarity (NLP + rule)
extracted_skills = []
vectorizer = TfidfVectorizer(lowercase=True, ngram_range=(1,3))  # thêm ngram để bắt "active directory"
vectorizer.fit(skills_db)  # CHỈ fit trên skills_db.3

skills_vec = vectorizer.transform(skills_db)
tokens_vec = vectorizer.transform([' '.join(tokens)])

for i, skill in enumerate(skills_db):
    sim = cosine_similarity(skills_vec[i], tokens_vec)[0][0]
    if sim > 0.2 :  # Rule: threshold
        extracted_skills.append(skill)

# Output unique skills
print(set(extracted_skills)) 

{'windows server', 'exchange', 'french', 'networking'}


# Tại sao bỏ TF-IDF + Cosine Similarity trong bài toán trích xuất Skill?

Trong demo ban đầu, phương pháp sau được sử dụng:

- TF-IDF Vectorizer
- Cosine Similarity

Tuy nhiên, cách tiếp cận này **không phù hợp cho bài toán trích xuất skill từ Job Description (JD) trong thực tế**.

---

## ❌ Vấn đề với TF-IDF + Cosine Similarity

### 1️⃣ Similarity được tính trên toàn bộ JD

- Job Description thường rất dài (vài trăm từ)
- Skill thường rất ngắn (1–3 từ)

Ví dụ:

- Skill: `active directory`
- JD: cả một đoạn mô tả dài

Khi tính cosine similarity giữa:
- vector của skill
- vector của toàn bộ JD

→ Giá trị similarity thường chỉ khoảng:
0.2 – 0.4

➡️ Rất khó đặt threshold hợp lý để quyết định skill có xuất hiện hay không.

---

### 2️⃣ Dễ bị nhiễu bởi các từ phổ biến

Các từ xuất hiện nhiều trong JD như:

- experience
- team
- knowledge
- responsible

Những từ này có trọng số TF-IDF cao và xuất hiện dày đặc → **làm loãng vector**.

➡️ Skill quan trọng dễ bị “chìm” trong không gian vector.

---

### 3️⃣ Không xử lý tốt các biến thể chuỗi

TF-IDF + cosine similarity **không linh hoạt** với các trường hợp:

| Trường hợp | Kết quả |
|-----------|--------|
| `Active Directory` vs `Directory Active` | Similarity thấp |
| Sai chính tả (`Active Diractory`) | Gần như không match |
| Viết tắt (`AD`) | Không match |

➡️ Phương pháp này không xử lý tốt:
- Thứ tự từ
- Sai chính tả ( trong truong hop cua Active Directory)
- Biến thể ngôn ngữ ( do csdl skill con han che va khong ho tro variant )

---

### 4️⃣ Hiệu năng kém khi danh sách skill lớn

Khi `skills_db` có hàng nghìn skill:

- Phải transform toàn bộ JD
- Phải transform từng skill
- Phải tính cosine similarity cho từng cặp

➡️ Tốn thời gian và không phù hợp cho hệ thống real-time.

---

## ✅ Tiep can voi phuong phap Exact Match + Fuzzy Matching tốt hơn?

---

### ✅ Exact Match – Bắt chính xác tuyệt đối

- Nếu skill xuất hiện **nguyên văn** trong JD → bắt ngay
- Không cần threshold
- Không có false positive

➡️ Độ chính xác gần như 100%.

---

### ✅ Fuzzy Matching – Xử lý biến thể cực tốt

Fuzzy matching (RapidFuzz) cho phép:

- Sai chính tả:
  - `Activ Directory` → `Active Directory`
- Thứ tự từ khác nhau:
  - `Directory Active` ↔ `Active Directory`
- Kết hợp synonym map:
  - `AD` → `Active Directory`

➡️ Phù hợp với JD thực tế (dữ liệu bẩn).

---
Fuzzy Matching (RapidFuzz): Không dùng vector, mà dùng string distance (Levenshtein/ratio trên token sorted). Scan sliding window (n-gram) trên JD để so trực tiếp với skill.
Khác biệt: Tập trung vào similarity chuỗi (bỏ qua thứ tự, bắt typo tốt như "Diractory" → 94% match "Directory"). Nhanh, không cần fit vector, nhưng thiếu context (không bắt synonym trừ khi bạn add variant vào DB).

### 🧠 Dễ kiểm soát và debug

- Biết rõ:
  - Skill nào được match
  - Match bằng exact hay fuzzy
  - Điểm similarity bao nhiêu

➡️ Dễ tinh chỉnh threshold và logic.

---

### 🎯 Độ chính xác thực tế cao (90–95%)

- Đây là cách mà **đa số hệ thống ATS thương mại** đang sử dụng
- Đã được kiểm chứng trong các hệ thống tuyển dụng thực tế

---

## 🧪 So sánh trực tiếp trên JD mẫu


In [9]:

from rapidfuzz import fuzz, process
from skillNer.general_params import SKILL_DB
import re

def normalize(text):
    return re.sub(r'\s+', ' ', text.strip().lower())

def extract_skills(jd_text, skills_db, threshold=85):
    jd_normalized = normalize(jd_text)
    tokens = jd_normalized.split()
    extracted = set()

    for skill in skills_db:
        skill_norm = normalize(skill)
        skill_tokens = skill_norm.split()
        k = len(skill_tokens)

        for i in range(len(tokens) - k + 1):
            window = " ".join(tokens[i:i+k])
            ratio = fuzz.ratio(skill_norm, window)

            if ratio >= threshold:
                extracted.add(skill)
                break

    return extracted

extract_skills(sample_jd,skills_db) 

{'Active Diractory',
 'english',
 'exchange',
 'french',
 'networking',
 'ticketing systems'}

In [12]:
import spacy
from spacy.matcher import PhraseMatcher
from skillNer.general_params import JOB_DB
from skillNer.skill_extractor_class import SkillExtractor

# Load model spaCy (chỉ làm 1 lần)
nlp = spacy.load("en_core_web_lg")

# Khởi tạo SkillExtractor
skill_extractor = SkillExtractor(nlp, JOB_DB, PhraseMatcher)

# Trích xuất
annotations = skill_extractor.annotate(sample_jd)

# In kết quả sạch (không cần import extract_skills nữa)
print("SkillNER extracted skills:\n")
full_matches = annotations['results']['full_matches']
ngram_scored = annotations['results']['ngram_scored']

all_skills = full_matches +ngram_scored

for item in all_skills:
    skill_name = item['doc_node_value']
    score = item['score']
    skill_type = item.get('type', 'N/A')  # một số có type như 'fullUni', 'lowForm'...
    print(f"- {skill_name} (score: {score:.2f}, type: {skill_type})")

ImportError: cannot import name 'JOB_DB' from 'skillNer.general_params' (c:\Users\trand\AppData\Local\Programs\Python\Python310\lib\site-packages\skillNer\general_params.py)

# Trong bài toán trích xuất skill từ Job Description (JD), có hai loại skill:

Explicit skills (skill được nói trực tiếp, rõ ràng)
→ Dễ bắt bằng các phương pháp đơn giản.
Implicit skills (skill ngầm hiểu, không được viết trực tiếp)
→ Rất khó bắt nếu chỉ dùng exact/fuzzy matching hoặc rule-based.

Câu đó muốn nói: Các phương pháp chúng ta đang thảo luận (Exact Match, Fuzzy Matching với RapidFuzz, SkillNER, TF-IDF) đều rất yếu trong việc bắt implicit skills. Nếu bạn thực sự cần bắt loại này thì phải dùng các mô hình BERT-based NER fine-tuned.
Ví dụ minh họa rõ nhất
Giả sử JD có câu:
"You will be the first point of contact for users reporting incidents, handling inquiries via phone, email, or ticketing system. Responsibilities include diagnosing and resolving hardware/software issues quickly, escalating complex problems when needed, and ensuring high customer satisfaction."
Explicit skills (dễ bắt):

ticketing system / ticketing systems
hardware
software

→ Exact/Fuzzy/SkillNER bắt được dễ dàng.
Implicit skills (khó bắt):

Help desk support hoặc IT support
(vì mô tả đúng công việc của help desk, nhưng không viết trực tiếp từ “help desk”)
Troubleshooting
(câu “diagnosing and resolving issues” chính là định nghĩa của troubleshooting, nhưng không dùng từ đó)
Customer service hoặc Customer support
(“ensuring high customer satisfaction”, “remain calm with frustrated users” → rõ ràng cần kỹ năng phục vụ khách hàng)
Incident management
(“reporting incidents”, “escalating complex problems” → đúng quy trình incident management trong ITIL)

→ Các phương pháp Exact/Fuzzy/SkillNER không bắt được những skill này vì:

Chúng chỉ tìm chuỗi gần giống với skill trong database.
Không hiểu ngữ nghĩa sâu, không suy luận được “diagnosing and resolving issues” = “troubleshooting”.

In [11]:



# Tạo PhraseMatcher riêng cho custom
custom_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")  # Case-insensitive
patterns = list(nlp.pipe(skills_db))  # Tạo doc patterns
custom_matcher.add("CUSTOM_SKILLS", patterns)

def extract_with_custom(text):
    # Trích xuất bằng SkillNER mặc định
    annotations = skill_extractor.annotate(text)
    
    # Thêm custom matching
    doc = nlp(text.lower())  # Hoặc text gốc nếu muốn giữ case
    custom_matches = custom_matcher(doc)
    
    custom_extracted = [doc[start:end].text for match_id, start, end in custom_matches]
    
    # Merge kết quả (loại duplicate)
    default_skills = [item['doc_node_value'] for item in annotations['results']['full_matches'] + annotations['results']['ngram_scored']]
    all_skills = default_skills +custom_extracted
    
    return all_skills

# Test với JD mẫu
all_skills = extract_with_custom(sample_jd)
for item in all_skills:
    print( item ) 

# → Sẽ bắt thêm "toeic", "devops", "kubernetes" dù không có trong SKILL_DB gốc

service desk
customer satisfaction
microsoft window
active directory
team orient
reporting incidents
inquiries
complex problems
ad
exchange server
basic networking
multitask
french
adapt
exchange
networking
french
